# Linear Classification

In this lab you will implement parts of a linear classification model using the r**egularized empirical risk minimization** principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [4]:
import urllib.request # 웹에서 데이터를 다운로드하는데 사용되는 라이브러리
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

#%load_ext autoreload # error
#%autoreload 2 # error

#get_ipython().run_line_magic('load_ext', 'autoreload') # IPython 환경에서 모듈을 자동으로 다시 로드하는 확장을 활성화
#get_ipython().run_line_magic('autoreload', '2') # 모든 모듈이 각 셀이 실행될 때마다 자동으로 다시 로드되도록 설정. 이는 코드 개발 중에 변경사항을 즉시 반영하도록 도와줌.

# 이를 통해 변경사항이 있는 셸이 자동으로 실행되어서 매번 누락된 셸을 찾아서 로드할 필요가 없어진다. 

testfile = urllib.request.URLopener() # URLopener 클래스의 인스턴스를 생성, 웹에서 파일을 다운로드하는데 사용
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")
# 지정된 URL로부터 파일을 로컬 시스템에 저장한다. 첫 번째 인자는 URL이고, 두 번째 인자는 로컬 파일 이름이다.

df_train = pd.read_csv('SPECT.train',header=None) # 다운된 파일을 읽어서 pandas dataframe으로 저장
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values # value 속성을 사용하여 데이터프레임의 값만을 numpy 배열로 반환
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

- 람다는 trade-off parameter로써, Empirical Risk과 Regularizer 간의 균형을 조절하는 하이퍼파라미터이다. λ 값이 크면 모델의 복잡도를 줄이고, 값이 작으면 모델이 데이터에 더 잘 적합하도록 한다.

Try to understand each step of the learning algorithm and comment each line.

Barzilai-Borwein method의 핵심 아이디어는 학습률(또는 스텝 사이즈)을 동적으로 조정하여 각 반복에서 이전 그래디언트 정보를 활용하는 것입니다.

- loss function은 모델의 예측 값과 실제 값 간의 차이를 정량화하는 함수다. 손실 함수의 목적은 이 차이를 최소화하는 방향으로 모델을 학습시키는 것이다.
- Gradient는 손실 함수의 매개변수에 대한 미분이다. 이는 손실 함수의 기울기를 나타내며, 가중치를 조정하여 손실을 최소화하는 방향을 제공한다.


In [6]:
# check the shape of the data

print(X_train.shape, y_test.shape)
print(X_train[0:5])
print(y_train)
w = np.random.randn(X_train.shape[1])
print(w.shape)

(80, 22) (187,)
[[0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1]
 [1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
(22,)


In [7]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200  # 최대 반복 횟수
    e  = 0.001      # 알고리즘이 종료되는 기준으로 사용한 error의 threshold
    alpha = 1.      # 초기 step size를 1로 설정

    w = np.random.randn(X.shape[1]); # weight 벡터 w를 랜덤하게 초기화. w의 dimension은 features의 개수와 같다.
    
    for k in np.arange(max_iter): # 200번 반복
        h = np.dot(X,w)         # predicted value h를 계산, 주어진 데이터 X와 weight 벡터 w의 내적으로 계산 (np.dot 함수는 두 배열의 dot product를 계산)
        l,lg = loss(h, y)       # predicted value h와 실제 레이블 y를 이용하여 loss L(wk)와 gradient ∇L(wk)를 계산한다. 
        #print ('loss: {}'.format(np.mean(l))) 
        r,rg = reg(w, lbda)     # Regularization term R(wk)와 그 gradient ∇R(wk)를 계산한다.
        g = np.dot(X.T,lg) + rg # loss의 gradient와 regularization의 gradient를 더하여 최종 gradient를 계산한다.
                                # X.T는 X의 전치행렬을 의미한다.
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) # Barzilai-Borwein 방법을 사용해 동적으로 step size α를 조절.
        w = w - alpha * g       # weight 벡터 w를 업데이트한다. 마이너스해서 그라디언트의 반대 방향으로 이동한다. 
        if (np.linalg.norm(alpha * g) < e): # 만약 gradient의 크기가 threshold보다 작아지면 알고리즘을 종료한다.
            break
        g_old = g # 사용한 그래디언트를 저장한다.
    return w

`g = np.dot(X.T,lg) + rg`는 왜 할까? 자 일단 gradient는 loss + regularizor이다. `np.dot(X.T,lg)`는 이 loss gradient를 weight에 대해 내적을 계산한다. 

단일 인스턴스의 gradient는 다음과 같이 정의된다.

$$\nabla_{x_i} L(\theta) = \frac{\partial}{\partial \theta} \ell(f_\theta(x_i), y_i) + \frac{\lambda}{n} \frac{\partial}{\partial \theta} \Omega(\theta)
$$

`reg()`함수에서 이미 weight를 받아서 weight $\theta$에 연관된 regularizer를 계산했고, loss에 대해서는 따로 이 weight에 대해 계산을 해줘야하 하므로 아래와 같은 과정을 거쳐준다.


$\frac{\partial l}{\partial w} = \frac{\partial l}{\partial h}\cdot \frac{\partial h}{\partial w}$이고, 이것은
$= lg \cdot \frac{\partial h}{\partial w}$이기 때문이다. 또 여기서 $\frac{\partial h}{\partial w}$는 x와 같다.


따라서, gradient를 계산하기 위해, $x^T \cdot lg$를 한다. 

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [8]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    l = np.maximum(0, 1 - h*y) # Hinge loss
    g = np.zeros_like(y)
    g[l > 0] = -y[l > 0]
    
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [9]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    ##################
    r = lbda / 2 * np.dot(w.T,w) # L2 regularization
    g = lbda * w
    
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [10]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    ##################
    if len(X.shape) == 1:
        preds = np.dot(w,X)
    else:
        preds = np.dot(X,w) 
    
    preds = np.sign(preds)
    preds[preds == 0] = -1 # -1 if 0
    
    return preds



### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [15]:
##################
#INSERT CODE HERE#
##################

def accuracy(y,pred):
    count = 0
    for i in range(len(y)):
        if y[i] == pred[i]:
            count += 1
    return count / len(y)

y_train_adj = np.copy(y_train)
y_test_adj = np.copy(y_test)

y_train_adj[y_train_adj == 0] = -1
y_test_adj[y_test_adj == 0] = -1


lambdas = [0.01, 0.1, 1, 10, 100]

for lbda in lambdas:
    w = learn_reg_ERM(X_train, y_train_adj, lbda)
    
    pred_train = predict(w, X_train)
    pred_test = predict(w, X_test)
    
    train_accuracy = accuracy(y_train_adj, pred_train)
    test_accuracy = accuracy(y_test, pred_test)
    
    print(f'lambda: {lbda}')

    print(f'train data accuracy: {train_accuracy * 100:.2f}%')
    print(f'test data accuracy: {test_accuracy * 100:.2f}% \n')


lambda: 0.01
train data accuracy: 87.50%
test data accuracy: 64.17% 

lambda: 0.1
train data accuracy: 85.00%
test data accuracy: 69.52% 

lambda: 1
train data accuracy: 62.50%
test data accuracy: 89.84% 

lambda: 10
train data accuracy: 66.25%
test data accuracy: 86.10% 

lambda: 100
train data accuracy: 62.50%
test data accuracy: 89.84% 



As the tradeoff parameter lambda value increases, the model's accuracy on the test data increases, and when the value is small, the model fits the training data better.

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

##################
#INSERT CODE HERE#
##################

## Dicision tree
dt = DecisionTreeClassifier(criterion='entropy', max_depth=1) 
dt.fit(X_train, y_train)
dt_predicted_train = dt.predict(X_train)
dt_predicted_test = dt.predict(X_test)

dt_train_accuracy = accuracy(y_train, dt_predicted_train)
dt_test_accuracy = accuracy(y_test, dt_predicted_test)
print(f'Decision Tree accuracy for train data: {dt_train_accuracy * 100:.2f}%')
print(f'Decision Tree accuracy for test data: {dt_test_accuracy * 100:.2f}% \n')

## Random Forest
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=1)
rf.fit(X_train, y_train)
rf_predicted_train = rf.predict(X_train)
rf_predicted_test = rf.predict(X_test)

rf_train_accuracy = accuracy(y_train, rf_predicted_train)
rf_test_accuracy = accuracy(y_test, rf_predicted_test)
print(f'Random Forest accuracy for train data: {rf_train_accuracy * 100:.2f}%')
print(f'Random Forest accuracy for test data: {rf_test_accuracy * 100:.2f}% \n')

# Linear classification with ERM
w = learn_reg_ERM(X_train, y_train_adj, 5)

lc_pred_train = predict(w, X_train)
lc_pred_test = predict(w, X_test)

lc_train_accuracy = accuracy(y_train_adj, lc_pred_train)
lc_test_accuracy = accuracy(y_test, lc_pred_test)

print(f'Linear classification with ERM accuracy for train data: {lc_train_accuracy * 100:.2f}%')
print(f'Linear classification with ERM accuracy for test data: {lc_test_accuracy * 100:.2f}% \n')

print(y_test)


Decision Tree accuracy for train data: 72.50%
Decision Tree accuracy for test data: 61.50% 

Random Forest accuracy for train data: 77.50%
Random Forest accuracy for test data: 68.45% 

Linear classification with ERM accuracy for train data: 70.00%
Linear classification with ERM accuracy for test data: 83.96% 

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
